### Load tensorflow

In [0]:
import tensorflow as tf

In [0]:
tf.__version__

'1.13.1'

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
tf.enable_eager_execution()

### Collect Data

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/gdrive/My Drive/Great Learning/Neural Networks I/Lab Solution/prices.zip')

### Check all columns in the dataset

In [0]:
data.columns

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date','symbol'], axis=1, inplace=True)

In [0]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [0]:
df = data.loc[:1000,:]

X = df[['open','close','low', 'high']]
y = df['volume']/1000000

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=.3, random_state=42)

In [0]:
import numpy as np

train_x = np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y = np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

### Normalize the data

In [0]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer()
train_x = transformer.fit_transform(train_x)
test_x = transformer.transform(test_x)

## Building the Model in tensorflow

1.Define Weights and Bias

In [0]:
#We are initializing weights and Bias with Zero
w = tf.random_normal(shape=(4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration

In [0]:
for i in range(100):
    
    w, b = train(train_x, train_y, w, b)
    print('Current Training Loss on iteration', i, loss(train_y, prediction(train_x, w, b)).numpy())

('Current Training Loss on iteration', 0, 2.6889987)
('Current Training Loss on iteration', 1, 2.6795757)
('Current Training Loss on iteration', 2, 2.6708918)
('Current Training Loss on iteration', 3, 2.6628883)
('Current Training Loss on iteration', 4, 2.6555126)
('Current Training Loss on iteration', 5, 2.6487148)
('Current Training Loss on iteration', 6, 2.64245)
('Current Training Loss on iteration', 7, 2.6366765)
('Current Training Loss on iteration', 8, 2.6313555)
('Current Training Loss on iteration', 9, 2.6264515)
('Current Training Loss on iteration', 10, 2.6219325)
('Current Training Loss on iteration', 11, 2.6177673)
('Current Training Loss on iteration', 12, 2.613929)
('Current Training Loss on iteration', 13, 2.6103914)
('Current Training Loss on iteration', 14, 2.607131)
('Current Training Loss on iteration', 15, 2.6041265)
('Current Training Loss on iteration', 16, 2.6013577)
('Current Training Loss on iteration', 17, 2.5988054)
('Current Training Loss on iteration', 18,

### Get the shapes and values of W and b

In [0]:
#Check Weights and shape
print('Weights Shape:', w.shape)
print('Weights:', w.numpy())

('Weights Shape:', TensorShape([Dimension(4), Dimension(1)]))
('Weights:', array([[1.7035644 ],
       [1.8509313 ],
       [0.49900162],
       [1.0767553 ]], dtype=float32))


In [0]:
#Check Bias
print('Bias Shape', b.shape)
print('Bias:',b.numpy())

('Bias Shape', TensorShape([Dimension(1)]))
('Bias:', array([2.9207187], dtype=float32))


### Model Prediction on 1st Examples in Test Dataset

In [0]:
prediction(test_x[0:1],w,b).numpy()

array([[5.4912577]], dtype=float32)

In [0]:
test_y[0]

2.6424